In [4]:
import openai
import langchain
import pinecone 
# from langchain_pinecone import Pinecone
from langchain.document_loaders import PyPDFDirectoryLoader # for loading pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter  #to tokenize into chunks
from  langchain.embeddings.openai import OpenAIEmbeddings # chunks to vectores
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp

from dotenv import load_dotenv
load_dotenv(override=True)
import os


In [5]:

## Read document
def read_doc(diretory):
    file_loader = PyPDFDirectoryLoader(diretory)
    print(file_loader)
    documents= file_loader.load()
    return documents

docs=read_doc("documents/")
print('len',len(docs))
print(type(docs))
for doc in docs:
    print(doc)


len 58
<class 'list'>
page_content='GOVERNMENT OF INDIA
BUDGET 2023-2024
SPEECH
OF
NIRMALA SITHARAMAN
MINISTER OF FINANCE
February 1,  2023' metadata={'source': 'documents/budget_speech.pdf', 'page': 0, 'page_label': '1'}
page_content='' metadata={'source': 'documents/budget_speech.pdf', 'page': 1, 'page_label': '2'}
page_content='CONTENTS 
PART-A 
 Page No. 
 Introduction 1 
 Achievements since 2014: Leaving no one behind 2 
 Vision for Amrit Kaal – an empowered and inclusive economy 3 
 Priorities of this Budget 5 
i. Inclusive Development  
ii. Reaching the Last Mile 
iii. Infrastructure and Investment 
iv. Unleashing the Potential 
v. Green Growth 
vi. Youth Power  
vii. Financial Sector 
 
 
 
 
 
 
 
 
 
 Fiscal Management 
24 
PART B 
  
Indirect Taxes 27 
 Green Mobility  
 Electronics   
 Electrical   
 Chemicals and Petrochemicals   
 Marine products  
 Lab Grown Diamonds  
 Precious Metals  
 Metals  
 Compounded Rubber  
 Cigarettes  
  
Direct Taxes 30 
 MS

In [6]:

## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size,chunk_overlap):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc 

chunkData = chunk_data(docs=docs, chunk_overlap=50, chunk_size=800)
chunkData


[Document(metadata={'source': 'documents/budget_speech.pdf', 'page': 0, 'page_label': '1'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023'),
 Document(metadata={'source': 'documents/budget_speech.pdf', 'page': 2, 'page_label': '3'}, page_content='CONTENTS \nPART-A \n Page No. \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector \n \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management \n24 \nPART B \n  \nIndirect Taxes 27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Prec

In [7]:

print("Type: ",type(chunkData), "Len:", len(chunkData))
print(chunkData[2])



Type:  <class 'list'> Len: 141
page_content=' Rationalisation  
 Others  
 Personal Income Tax  
  
Annexures 35 
 Annexure to Part B of the Budget Speech 2023-24 
i. Amendments relating to Direct Taxes 
ii. Amendments relating to Indirect Taxes' metadata={'source': 'documents/budget_speech.pdf', 'page': 2, 'page_label': '3'}


In [ ]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

vectors=embeddings.embed_query("How are you?")

KeyError: 'OPEN_API_KEY'

In [ ]:

#Connect to Pinecone
#Connect to Pinecone
pc = pinecone.Pinecone(api_key=os.environ['PINECONE_API_KEY'])  # Use environment variable
# Connect to your index

pinceConeIndex=Pinecone.from_documents(documents=chunkData , embedding=embeddings, index_name="langchain")
print(pinceConeIndex)

In [ ]:

# Initialize the LLM with your API key
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)

# Load the QA chain using the 'stuff' chain type
chain = load_qa_chain(llm, chain_type="stuff")
# chain=load_qa_chain(llm,chain_type="stuff")

# Using Cosine similarity retrive mathcing results from vectorDB
def retrive_query(query,k=2):
    matching_results=pinceConeIndex.similarity_search(query,k=k)
    return matching_results
# Search Answers from Vector DB
def retrieve_answers(query):
    doc_search=retrive_query(query=query)
    print(doc_search)
    response=chain.run(input_documents=doc_search, question=query)
    return response
    
